In [1]:
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import subprocess


In [2]:
stopwords=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
punkts='''"#$%&\'()*+,-./:;<=>@[\\]^_`{|}~'''

In [15]:
def CorFilt(i):
    ps = PorterStemmer()
    
    buff=word_tokenize(i.lower().replace("\n","").replace("  "," ").replace("n't"," not"))
    buff2=""
    for j in pos_tag(buff):
        if j[-1]=='RB' and j[0]!="not":
            pass
        else:
            buff2+=j[0]+" "
    buff2=buff2.replace("not ","NOT")
    buff=word_tokenize(buff2.strip())
    ans=""
    for j in buff:
        if (j not in punkts) and (j not in stopwords):
            if j=="!":
                ans+=" XXEXCLMARK"
            elif j=="?":
                ans+=" XXQUESMARK"
            else:
                if j!="'s" and j!="``":
                    ans+=" "+ps.stem(j)
    return ans.strip()

In [13]:
import pickle
f=open("EmoVec","rb")
EmoVec=pickle.load(f)
f.close()

f=open("vectorizer","rb")
vectorizer=pickle.load(f)
f.close()

In [14]:
def EmowavE(sent,vectorizer=vectorizer,EmoVec=EmoVec,trans=True):
    
    transDict={'gu':'Gujarati',
               'hi':'Hindi'}
    # Translate from any language to english
    if trans:
        analysis = TextBlob(sent)
        if analysis.detect_language()!='en':
            try:
                print(f"\nInput text was in {transDict[analysis.detect_language()]}")
            except:
                print(f"\nInput text was not in English")
            print("\nTranslating...")
            output=subprocess.check_output(['trans','-b',sent])
            sent=output.decode('utf-8').strip()
            print(f"\nTranslation in English: {sent}")
        
    EmoBuff=vectorizer.transform([CorFilt(sent)])
    EmoDict={0:'anger',
             1:'disgust',
             2:'fear',
             3:'joy',
             4:'sadness'}
    return EmoDict[np.argmax([float(cosine_similarity(EmoBuff.reshape(-1,1).T,EmoVec[i].reshape(-1,1).T)) for i in range(EmoVec.shape[0])])]

In [17]:
print(f"\n\t>>> Emotion: {EmowavE('a perfectly parceled dead body')}")


	>>> Emotion: fear
